In [ ]:
import numpy as np
import scipy.signal
import matplotlib.pylab as plt
import matplotlib.animation
import IPython.display

np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)                 

# default values
R = 1

def figure_world(A, cmap='viridis'):
  global img
  fig = plt.figure()
  img = plt.imshow(A, cmap=cmap, interpolation="nearest", vmin=0)
  plt.title('world A')
  plt.close()
  return fig

def figure_asset(K, growth, cmap='viridis', K_sum=1, bar_K=False):
  global R
  K_size = K.shape[0];  K_mid = K_size // 2
  fig, ax = plt.subplots(1, 3, figsize=(14,2), gridspec_kw={'width_ratios': [1,1,2]})
  ax[0].imshow(K, cmap=cmap, interpolation="nearest", vmin=0)
  ax[0].title.set_text('kernel K')
  if bar_K:
    ax[1].bar(range(K_size), K[K_mid,:], width=1)
  else:
    ax[1].plot(range(K_size), K[K_mid,:])
  ax[1].title.set_text('K cross-section')
  ax[1].set_xlim([K_mid - R - 3, K_mid + R + 3])
  if K_sum <= 1:
    x = np.linspace(0, K_sum, 1000)
    ax[2].plot(x, growth(x))
  else:
    x = np.arange(K_sum + 1)
    ax[2].step(x, growth(x))
  ax[2].axhline(y=0, color='grey', linestyle='dotted')
  ax[2].title.set_text('growth G')
  return fig

def figure_asset_list(Ks, nKs, growth, kernels, use_c0=False, cmap='viridis', K_sum=1):
  global R
  K_size = Ks[0].shape[0];  K_mid = K_size // 2
  fig, ax = plt.subplots(1, 3, figsize=(14,2), gridspec_kw={'width_ratios': [1,2,2]})
  if use_c0:
    K_stack = [ np.clip(np.zeros(Ks[0].shape) + sum(K/3 for k,K in zip(kernels,Ks) if k['c0']==l), 0, 1) for l in range(3) ]
  else:
    K_stack = Ks[:3]
  ax[0].imshow(np.dstack(K_stack), cmap=cmap, interpolation="nearest", vmin=0)
  ax[0].title.set_text('kernels Ks')
  X_stack = [ K[K_mid,:] for K in nKs ]
  ax[1].plot(range(K_size), np.asarray(X_stack).T)
  ax[1].title.set_text('Ks cross-sections')
  ax[1].set_xlim([K_mid - R - 3, K_mid + R + 3])
  x = np.linspace(0, K_sum, 1000)
  G_stack = [ growth(x, k['m'], k['s']) * k['h'] for k in kernels ]
  ax[2].plot(x, np.asarray(G_stack).T)
  ax[2].axhline(y=0, color='grey', linestyle='dotted')
  ax[2].title.set_text('growths Gs')
  return fig

def figure_panels(As, Ks, cmap='viridis'):
  global img1, img2, img3, img4
  A_size = As[0].shape[0]
  K_stack = [ np.clip(np.zeros(Ks[0].shape) + sum(K/3 for k,K in zip(kernels,Ks) if k['c0']==l), 0, 1) for l in range(3) ]
  fig = plt.figure(figsize=(8,8), dpi=75, frameon=False)
  plt.subplot(221).set_axis_off();  img1 = plt.imshow(np.dstack(As), vmin=0)
  plt.subplot(222).set_axis_off();  img2 = plt.imshow(np.dstack([np.zeros([A_size, A_size])]*3), vmin=0)
  plt.subplot(223).set_axis_off();  img3 = plt.imshow(np.dstack([np.zeros([A_size, A_size])]*3), vmin=0)
  plt.subplot(224).set_axis_off();  img4 = plt.imshow(np.dstack(K_stack), vmin=0)
  fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0, hspace=0)
  plt.close()
  return fig


In [ ]:
# world size
size = 200
T = 10
# kernel radius
R = 5

# seed the random generator
np.random.seed(0)
A = np.random.rand(size, size)

K = np.ones((2*R+1, 2*R+1))
K[R, R] = 0
K = K / np.sum(K)

def update(i):
    global A
    U = scipy.signal.convolve2d(A, K, mode='same', boundary='wrap')
    A = np.clip(A + 1/T * growth(U), 0, 1)
    img.set_array(A)
    return img,

def growth(U):
    return 0 + ((U >= 0.12) & (U <= 0.15)) - ((U < 0.12)|(U > 0.15))

figure_asset(K, growth, bar_K=True)

In [ ]:
fig = figure_world(A)
IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update, frames=200, interval=20).to_jshtml())

In [ ]:
# The ring kernel
size = 200
T = 10
# kernel radius
R = 5
np.random.seed(0)
world = np.random.rand(size, size)

K = np.asarray([
    [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
    [1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1],
    [1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1],
    [1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
    [0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0],
    [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0]
])


K = K / np.sum(K)

def growth_func(U):
    return 0 + ((U >= 0.12) & (U <= 0.15)) - ((U < 0.12) | (U > 0.15))

def update(i):
    global world, img
    U = scipy.signal.convolve2d(world, K, mode='same', boundary='wrap')
    world = np.clip(world + 1/T * growth_func(U), 0, 1)
    img.set_array(world)
    return img,

figure_asset(K, growth_func, bar_K=True)
fig = figure_world(world)
IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update, frames=200, internal=20).to_jshtml())

In [ ]:
bell = lambda x, m, s: np.exp(-((x-m)/s)**2 / 2)
size = 256
T = 10
R = 10
np.random.seed(0)

World = np.random.rand(size, size)
D = np.linalg.norm(np.asarray(np.ogrid[-R:R, -R:R]) + 1) / R
K = (D < 1) * bell(D, 0.5, 0.15)
K = K / np.sum(K)

def growth(U):
    ''' smooth growth function '''
    m = 0.135
    s = 0.015
    return bell(U, m, s) * 2-1

def update(i):
    global World, img
    U = scipy.signal.convolve2d(World, K, mode='same', boundary='wrap')
    World = np.clip(World + 1/T * growth(U), 0, 1)
    img.set_array(A)
    return img,

figure_asset(K, growth)
fig = figure_world(World)
IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update, frames=200, interval=20).to_jshtml)

In [ ]:
np.random.seed(0)
A = np.random.randint(2, size=(64, 64))

In [ ]:
def update(i):
    global A
    U = sum(np.roll(A, (i, j), axis=(0, 1)) for i in (-1, 0, +1) for j in (-1, 0, +1) if (i != 0 or j != 0))
    A = (A & (U == 2)) | (U == 3)
    img.set_array(A)
    return img,